<a href="https://colab.research.google.com/github/Faisal-Al-Mamun/Boat-Types-Classification-PyTorch/blob/main/Boat_Types_Classification_%7C_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset: https://www.kaggle.com/clorichel/boat-types-recognition

Kaggle Notebook Link: https://www.kaggle.com/faisalalmamun007/boat-types-classification-pytorch

### Importing Libraries

In [ ]:
! pip install -q split-folders

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sn
import sklearn.metrics


import splitfolders

### Loading Dataset & Data Augmentation

In [ ]:
! mkdir data

In [ ]:
splitfolders.ratio("../input/boat-images", output="./data", seed=46, ratio=(.8, .2), group_prefix=None) # default values

Copying files: 1462 files [00:10, 143.13 files/s]


In [ ]:
transform = transforms.Compose([
    transforms.Resize([100,100]),
    transforms.RandomRotation(degrees=[-30,30]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform2 = transforms.Compose([
    transforms.Resize([100,100]),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
data_dir = './data/train'
classes = os.listdir(data_dir)
print(classes)
print("Total Class:",len(classes))

['cruise ship', 'kayak', 'freight boat', 'gondola', 'sailboat', 'ferry boat', 'paper boat', 'inflatable boat', 'buoy']
Total Class: 9


In [ ]:
test_ds = torchvision.datasets.ImageFolder('./data/val', transform=transform2)
train_ds = torchvision.datasets.ImageFolder('./data/train', transform=transform)

In [ ]:
print(len(train_ds))

1166


In [ ]:
print(len(test_ds))

296


In [ ]:
train_ds.classes

['buoy',
 'cruise ship',
 'ferry boat',
 'freight boat',
 'gondola',
 'inflatable boat',
 'kayak',
 'paper boat',
 'sailboat']

In [ ]:
# view one image shape of the dataset.
img, label = train_ds[10]
print(img.shape)

torch.Size([3, 100, 100])


### Make Dataset Iterable

In [ ]:
batch_size = 16

num_epochs = 25

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_ds, batch_size=batch_size, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_ds, batch_size=batch_size, shuffle=False)

### Create Model Class | CNN Model 1

In [ ]:
class CNNModelOne(nn.Module):
    def __init__(self):
        super(CNNModelOne, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()
        
        # Max pool 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 
        self.fc1 = nn.Linear(128 * 12 * 12, 1024) 
        self.lrelu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.40)
        
        self.fc2 = nn.Linear(1024, 512)
        self.lrelu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.30)
        
        self.fc3 = nn.Linear(512, 256)
        self.lrelu3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.25)
        
        self.fc4 = nn.Linear(256, 9)
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Convolution 3 
        out = self.cnn3(out)
        out = self.relu3(out)
        
        # Max pool 3 
        out = self.maxpool3(out)
        
        # Flatten
        out = torch.flatten(out, 1) # flatten all dimensions except the batch dimension

        # Linear function (readout)
        out = self.fc1(out)
        out = self.lrelu1(out)
        out = self.drop1(out)
        
        out = self.fc2(out)
        out = self.lrelu2(out)
        out = self.drop2(out)
        
        out = self.fc3(out)
        out = self.lrelu3(out)
        out = self.drop3(out)
        
        out = self.fc4(out)
        
        return out

### Instantiate Model Class

In [ ]:
model = CNNModelOne()

In [ ]:
model = model.cuda()

### Instantiate Loss Class

In [ ]:
criterion = nn.CrossEntropyLoss()

### Instantiate Optimizer Class

In [ ]:
learning_rate = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

Parameters In-Depth

In [ ]:
print(model)

CNNModelOne(
  (cnn1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=18432, out_features=1024, bias=True)
  (lrelu1): ReLU()
  (drop1): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (lrelu2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (lrelu3): ReLU()
  (drop3): Dropout(p=0.25, inplace=False)
  (fc4): Linear(in_features=256, out_features=9, bias=True)
)


In [ ]:
print(model.parameters())

print(len(list(model.parameters())))

<generator object Module.parameters at 0x7f1e5f1b91d0>
14


### Train Model


In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images.cuda())
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels.cuda())
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
 
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in test_loader:
         # Load images
        images = images.requires_grad_()
                
        # Forward pass only to get logits/output
        outputs = model(images.cuda())
                
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
                
        labels = labels.cuda() 
        # Total number of labels
        total += labels.size(0)
                
        # Total correct predictions
        correct += (predicted == labels).sum()
            
    accuracy = 100 * correct / total
            
    # Print Loss
    print('Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}'.format(epoch, iter, loss.item(), accuracy))

Epoch: 0. Iteration: 73. Loss: 1.687656044960022. Accuracy: 33.10810852050781
Epoch: 1. Iteration: 146. Loss: 1.4217536449432373. Accuracy: 32.09459686279297
Epoch: 2. Iteration: 219. Loss: 2.356048583984375. Accuracy: 34.797298431396484
Epoch: 3. Iteration: 292. Loss: 1.5733023881912231. Accuracy: 31.418920516967773
Epoch: 4. Iteration: 365. Loss: 1.7734320163726807. Accuracy: 36.486488342285156
Epoch: 5. Iteration: 438. Loss: 1.821069598197937. Accuracy: 35.135135650634766
Epoch: 6. Iteration: 511. Loss: 1.403670072555542. Accuracy: 35.135135650634766
Epoch: 7. Iteration: 584. Loss: 1.670939326286316. Accuracy: 38.85135269165039
Epoch: 8. Iteration: 657. Loss: 1.3008849620819092. Accuracy: 35.47297286987305
Epoch: 9. Iteration: 730. Loss: 1.6123889684677124. Accuracy: 40.878379821777344
Epoch: 10. Iteration: 803. Loss: 1.5276187658309937. Accuracy: 37.5
Epoch: 11. Iteration: 876. Loss: 1.5328289270401. Accuracy: 45.27027130126953
Epoch: 12. Iteration: 949. Loss: 1.8878424167633057. A